# export tiles for contextal layers


In [1]:
import ee
import geemap

In [ ]:
# Trigger the authentication flow.
# ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate(auth_mode='notebook')

In [3]:
# Initialize the library.
ee.Initialize()

In [7]:
def exportMapTask(asset: ee.Image, asset_name: str, gcbucket: str,
                #region: ee.Geometry, min_zoom: int, max_zoom: int,
                min_zoom: int, max_zoom: int, 
                env: str = 'staging', 
                key: str = 'year'):
    """
    Export the data to GEE.

    Parameters
    ----------
    asset : ee.Image
        The image to export.
    asset_name : str
        The name of the asset.
    gcbucket : str
        The name of the GCS bucket.
    geometry_collection : ee.GeometryCollection
        The geometry collection to export the data.
    
    Returns
    -------
    A task
    """
    
    
    return ee.batch.Export.map.toCloudStorage(
            image = asset,
            description = f'{asset_name}',
            path = f'{env}/tilesets/{asset_name}',
            bucket = gcbucket,
            minZoom = min_zoom,
            maxZoom = max_zoom,
            writePublicTiles = False,
            skipEmptyTiles = True,
            fileFormat = 'png'#,
            #region = region
            )

### Data Layer: Fishing intensity associated with mangroves
#### Visualize the data prior exporting it

In [8]:
gcs_bucket = 'mangrove_atlas'

dataset = f'fishing_intensity_mangroves'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
image = ee.Image("projects/mangrove-atlas-246414/assets/Fishing_Pressure_3857")

# exagerated image overviews for zoomed out views
#overviews = image_extent.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
visoptions =  {"palette": ['B7E6A5', 'ca5268'], "min": 1, "max": 2}

styled_image = image.sldStyle("""
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
            <ColorMapEntry color="#F3E79B" quantity="50" label="0 - 50"/> +
            <ColorMapEntry color="#FAC484" quantity="200" label=">50 - 200"/> +
            <ColorMapEntry color="#F0746E" quantity="700" label=">200 - 700"/> +
            <ColorMapEntry color="#B9257A" quantity="2000" label=">700 - 2000"/> +
            <ColorMapEntry color="#701A64" quantity="2000000" label=">2000"/> +
        </ColorMap>
    </RasterSymbolizer>
""")
# exagerated image overviews for zoomed out views
overviews = styled_image.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 2).selfMask()

Map = geemap.Map(center=( 3.3, 117.4), zoom=9, basemap='CartoDB.PositronNoLabels')

Map.addLayer(styled_image, {}, f"{dataset}", True, 1)
Map.addLayer(overviews, {}, f"{dataset}_overviews", True, 1)

Map

Map(center=[3.3, 117.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

#### Export to tiles from GEE to GCS

In [9]:
task = exportMapTask(overviews, f'{dataset}', gcs_bucket, 
                     1, 3)
# Create a list of tasks to export the map tiles
task2 = exportMapTask(styled_image, f'{dataset}', gcs_bucket, 
                     4, 12)
# execute tasks in batches due to gee limits
task.start()
task2.start()


### Data Layer: Bivalve enhancement values
#### Visualize the data prior exporting it

In [11]:
gcs_bucket = 'mangrove_atlas'

dataset = f'fishing_enhancement_bivalve'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
image = ee.Image("projects/mangrove-atlas-246414/assets/Bivalve_3857")

# exagerated image overviews for zoomed out views
#overviews = image_extent.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
visoptions =  {"palette": ['B7E6A5', 'ca5268'], "min": 1, "max": 2}

styled_image_bivalve = image.sldStyle("""
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
            <ColorMapEntry color="#fdefe2" quantity="0" label="0 - 50"/> +
            <ColorMapEntry color="#F4C28C" quantity="50" label=">50 - 70"/> +
            <ColorMapEntry color="#ED904E" quantity="7" label=">70 - 90"/> +
            <ColorMapEntry color="#D45E2B" quantity="90" label=">90 - 110"/> +
            <ColorMapEntry color="#9a3f1b" quantity="110" label=">110"/> +
        </ColorMap>
    </RasterSymbolizer>
""")
# exagerated image overviews for zoomed out views
overviews_bivalve = styled_image_bivalve.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 2).selfMask()

Map = geemap.Map(center=( 22.3, -105), zoom=9, basemap='CartoDB.PositronNoLabels')

Map.addLayer(styled_image_bivalve, {}, f"{dataset}", True, 1)
#Map.addLayer(overviews_bivalve, {}, f"{dataset}", True, 1)
#Map.addLayer(styled_image, {}, "Fishing pressure", True, 1)

Map

Map(center=[22.3, -105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

#### Export to tiles from GEE to GCS

In [12]:
task = exportMapTask(overviews_bivalve, f'{dataset}', gcs_bucket, 
                     1, 3)
# Create a list of tasks to export the map tiles
task2 = exportMapTask(styled_image_bivalve, f'{dataset}', gcs_bucket, 
                     4, 12)
# execute tasks in batches due to gee limits
task.start()
task2.start()


### Data Layer: Crab enhancement values
#### Visualize the data prior exporting it

In [13]:
gcs_bucket = 'mangrove_atlas'

dataset = f'fishing_enhancement_crab'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
image = ee.Image("projects/mangrove-atlas-246414/assets/Crab_3857")
# exagerated image overviews for zoomed out views
#overviews = image_extent.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
#visoptions =  {"palette": ['B7E6A5', 'ca5268'], "min": 1, "max": 2}

styled_image_crab = image.sldStyle("""
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
            <ColorMapEntry color="#EFFAEC" quantity="0." label="0 - 0.5"/> +
            <ColorMapEntry color="#C2E2B8" quantity="0.5" label=">0.5 - 0.6"/> +
            <ColorMapEntry color="#84C27C" quantity="0.6" label=">0.6 - 1"/> +
            <ColorMapEntry color="#529B58" quantity="1" label=">1 - 100"/> +
            <ColorMapEntry color="#2E6B34" quantity="100" label=">100"/> +
        </ColorMap>
    </RasterSymbolizer>
""")
# exagerated image overviews for zoomed out views
overviews_crab = styled_image_crab.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 2).selfMask()

Map = geemap.Map(center=( 22.3, -105), zoom=9, basemap='CartoDB.PositronNoLabels')

Map.addLayer(styled_image_crab, {}, f"{dataset}", True, 1)
#Map.addLayer(overviews_crab, {}, f"{dataset}", True, 1)

Map

Map(center=[22.3, -105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

#### Export to tiles from GEE to GCS

In [14]:
task = exportMapTask(overviews_crab, f'{dataset}', gcs_bucket, 
                     1, 3)
# Create a list of tasks to export the map tiles
task2 = exportMapTask(styled_image_crab, f'{dataset}', gcs_bucket, 
                     4, 12)
# execute tasks in batches due to gee limits
task.start()
task2.start()


### Data Layer: Finfish enhancement values
#### Visualize the data prior exporting it

In [15]:
gcs_bucket = 'mangrove_atlas'

dataset = f'fishing_enhancement_fish'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False);
# layer to export
image = ee.Image("projects/mangrove-atlas-246414/assets/Fish_3857")
# exagerated image overviews for zoomed out views
#overviews = image_extent.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
#visoptions =  {"palette": ['B7E6A5', 'ca5268'], "min": 1, "max": 2}

styled_image_fish = image.sldStyle("""
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
            <ColorMapEntry color="#f0f2fb" quantity="0." label="0 - 40"/> +
            <ColorMapEntry color="#c2d6db" quantity="40" label=">40 - 80"/> +
            <ColorMapEntry color="#7bacd1" quantity="80" label=">80 - 105"/> +
            <ColorMapEntry color="#4b7baf" quantity="105" label=">105 - 115"/> +
            <ColorMapEntry color="#234f97" quantity="115" label=">115"/> +
        </ColorMap>
    </RasterSymbolizer>
""")
# exagerated image overviews for zoomed out views
overviews_fish = styled_image_fish.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 2).selfMask()

Map = geemap.Map(center=( 18.2, -88), zoom=9, basemap='CartoDB.PositronNoLabels')

Map.addLayer(styled_image_fish, {}, f"{dataset}", True, 1)
#Map.addLayer(overviews_fish, {}, f"{dataset}", True, 1)

Map

Map(center=[18.2, -88], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

#### Export to tiles from GEE to GCS

In [16]:
task = exportMapTask(overviews_fish, f'{dataset}', gcs_bucket, 
                     1, 3)
# Create a list of tasks to export the map tiles
task2 = exportMapTask(styled_image_fish, f'{dataset}', gcs_bucket, 
                     4, 12)
# execute tasks in batches due to gee limits
task.start()
task2.start()


### Data Layer: Shrimp enhancement values
#### Visualize the data prior exporting it

In [17]:
gcs_bucket = 'mangrove_atlas'

dataset = f'fishing_enhancement_shrimp'
# Region to export
region = ee.Geometry.Polygon([[[-180, 33],[-180, -34],[180, -34],[180, 33]]], None, False)
# layer to export
image = ee.Image("projects/mangrove-atlas-246414/assets/Shrimp_3857")
# exagerated image overviews for zoomed out views
#overviews = image_extent.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 3).selfMask()
# visualization parameters
#visoptions =  {"palette": ['B7E6A5', 'ca5268'], "min": 1, "max": 2}

styled_image_shrimp = image.sldStyle("""
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
            <ColorMapEntry color="#fcede4" quantity="0." label="0 - 170"/> +
            <ColorMapEntry color="#f0b7bc" quantity="170" label=">170 - 220"/> +
            <ColorMapEntry color="#e970a2" quantity="220" label=">220 - 240"/> +
            <ColorMapEntry color="#b73088" quantity="240" label=">240 - 260"/> +
            <ColorMapEntry color="#6e1375" quantity="260" label=">260"/> +
        </ColorMap>
    </RasterSymbolizer>
""")
# exagerated image overviews for zoomed out views
overviews_shrimp = styled_image_shrimp.resample('bilinear').unmask().focalMode(1, 'circle', 'pixels', 2).selfMask()

Map = geemap.Map(center=( -3.2, 116.1), zoom=9, basemap='CartoDB.PositronNoLabels')

Map.addLayer(styled_image_shrimp, {}, f"{dataset}", True, 1)
#Map.addLayer(overviews_shrimp, {}, f"{dataset}", True, 1)

Map

Map(center=[-3.2, 116.1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

#### Export to tiles from GEE to GCS

In [18]:
task = exportMapTask(overviews_shrimp, f'{dataset}', gcs_bucket, 
                     1, 3)
# Create a list of tasks to export the map tiles
task2 = exportMapTask(styled_image_shrimp, f'{dataset}', gcs_bucket, 
                     4, 12)
# execute tasks in batches due to gee limits
task.start()
task2.start()
